In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/Graduation Project/'

!unzip /content/drive/MyDrive/Graduation\ Project/data.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/Camera 1/train/c7/844000000.jpg  
  inflating: data/Camera 1/train/c7/845000000.jpg  
  inflating: data/Camera 1/train/c7/846000000.jpg  
  inflating: data/Camera 1/train/c7/847000000.jpg  
  inflating: data/Camera 1/train/c7/848000000.jpg  
  inflating: data/Camera 1/train/c7/862000000.jpg  
  inflating: data/Camera 1/train/c7/863000000.jpg  
  inflating: data/Camera 1/train/c7/864000000.jpg  
  inflating: data/Camera 1/train/c7/865000000.jpg  
  inflating: data/Camera 1/train/c7/866000000.jpg  
  inflating: data/Camera 1/train/c7/867000000.jpg  
  inflating: data/Camera 1/train/c7/868000000.jpg  
  inflating: data/Camera 1/train/c7/869000000.jpg  
  inflating: data/Camera 1/train/c7/870000000.jpg  
  inflating: data/Camera 1/train/c7/871000000.jpg  
  inflating: data/Camera 1/train/c7/872000000.jpg  
  inflating: data/Camera 1/train/c7/873000000.jpg  
  inflating: data/Camera 1/train/c7/874000000.jpg  
  inflating: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim  
import torchvision.transforms as transforms
import torchvision
import os
from torchvision.io import decode_jpeg
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets,models
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision.io import read_image

In [ ]:
NUM_CLASSES = 10
BATCH_SIZE = 32

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
class CreateDataset(Dataset):
    def __init__(self, df,transform=False):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):   
        img_path = self.df.iloc[index, 0]
        image = read_image(img_path) / 255.0
        label = self.df.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
transformers = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_df = pd.read_csv(PATH + "train.csv")
val_df = pd.read_csv(PATH + "val.csv")
test_df = pd.read_csv(PATH + "test.csv")

train_dataset=CreateDataset(train_df.append(test_df, ignore_index=True), transformers)
# test_dataset=CreateDataset(test_df, transformers)
val_dataset=CreateDataset(val_df, transformers)

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE,shuffle=False)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE,shuffle=False)

In [ ]:
# !pip install pytorch-lightning torchmetrics

In [ ]:
import pytorch_lightning as pl
import torchmetrics
from torch import nn


class Model(pl.LightningModule):
    def __init__(self, output_units, dropout, learning_rate, freeze_base=False):
        super().__init__()
        self.base_model = torchvision.models.mobilenet_v2(pretrained=True)
        
        if freeze_base:
            for layer in self.base_model.features:
              for param in layer:
                param.requires_grad = False
                
        self.base_model.classifier[0] = nn.Dropout(p=dropout)
        self.base_model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=output_units)
        
        self.criterion = nn.CrossEntropyLoss()
        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()

        self.learning_rate = learning_rate
        self.save_hyperparameters()
        
    def forward(self, input_data):
        return self.base_model(input_data)

    def training_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        loss = self.criterion(preds, targets)
        self.log('train_loss', loss)
        self.train_acc(preds, targets)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=False, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        loss = self.criterion(preds, targets)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.val_acc(preds, targets)
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True)
        
    def test_step(self, batch, batch_nb):
        self.validation_step(batch, batch_nb)
        
    def predict_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        return torch.argmax(preds, dim=1)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

In [ ]:
model = Model(NUM_CLASSES, 0.2, 1e-5)
callbacks = [
    pl.callbacks.ModelCheckpoint(monitor='val_acc', dirpath=PATH, verbose=True, mode='max', filename='mobile_net'),
    pl.callbacks.EarlyStopping(monitor='val_acc', patience=15, verbose=True, mode='max')
]

trainer = pl.Trainer(max_epochs=100, callbacks=callbacks, gpus=1)    
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | base_model | MobileNetV2      | 2.2 M 
1 | criterion  | CrossEntropyLoss | 0     
2 | train_acc  | Accuracy         | 0     
3 | val_acc    | Accuracy         | 0     
------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.947     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:623: UserWarning: Checkpoint directory /content/drive/MyDrive/Graduation Project exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Metric val_acc improved. New best score: 0.457
Epoch 0, global step 364: val_acc reached 0.45730 (best 0.45730), saving model to "/content/drive/MyDrive/Graduation Project/mobile_net.ckpt" as top 1


...

Metric val_acc improved by 0.004 >= min_delta = 0.0. New best score: 0.783
Epoch 11, global step 4379: val_acc reached 0.78256 (best 0.78256), saving model to "/content/drive/MyDrive/Graduation Project/mobile_net.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Metric val_acc improved by 0.005 >= min_delta = 0.0. New best score: 0.788
Epoch 12, global step 4744: val_acc reached 0.78790 (best 0.78790), saving model to "/content/drive/MyDrive/Graduation Project/mobile_net.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 13, global step 5109: val_acc was not in top 1
